In [ ]:
#| default_exp logs.services.console

In [ ]:
#| exporti
import json
from typing import List

from dc_logger.client.base import ServiceHandler, ServiceConfig, LogEntry


In [ ]:
#| exporti

from dc_logger.client.base import ServiceConfig, OutputMode
from dataclasses import dataclass

In [ ]:
#| export

@dataclass
class Console_ServiceConfig(ServiceConfig):
    """Console-specific log configuration"""
    
    output_mode: OutputMode = "console"

    def validate_config(self) -> bool:
        return True

In [ ]:
#| export

class ConsoleHandler(ServiceHandler):
    """Handler for console output"""

    async def _write_json(self, entry: LogEntry) -> dict:
        message = json.dumps(entry.to_dict(), indent=2, default=str)
        print(message)
        return message


    async def _write_text(self, entry: LogEntry) -> str:
        message = f"[{entry.timestamp}] {entry.level.value} {entry.logger}: {entry.message}"
        print(message)
        return message
                

    async def write(self, entry: LogEntry, output_type) -> bool:
        """Write entries to console"""
        
        try:
            if output_type == "json":
                return self._write_json(entry)

            if output_type == "text":
                return self._write_text(entry)
                
        except Exception as e:
            print(f"Error writing to console: {e}")
            return False

    async def flush(self) -> bool:
        """Console output doesn't need flushing"""
        return True


In [ ]:
class ToCSV(HandlerInstance):
    output_type : Literal["str", 'json']

    def write(self):
        self.handler.write(... output_type = self.output_type)

In [ ]:
Console_ServiceConfig()